In [1]:
import os
import zarr
from glob import glob

import numpy as np
import xarray as xr

In [2]:
import pandas as pd 

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
def detrend_linear(da, dim="time"):
    """
    Remove a best-fit linear trend along `dim` for each grid point.
    Uses an index-based time axis (0..N-1) to avoid datetime scaling issues.
    """
    t = xr.DataArray(np.arange(da.sizes[dim]), dims=dim, coords={dim: da[dim]})

    valid = np.isfinite(da)
    t_valid = t.where(valid)
    da_valid = da.where(valid)

    t_mean = t_valid.mean(dim, skipna=True)
    y_mean = da_valid.mean(dim, skipna=True)

    cov = ((t_valid - t_mean) * (da_valid - y_mean)).mean(dim, skipna=True)
    var = ((t_valid - t_mean) ** 2).mean(dim, skipna=True)

    slope = cov / var
    intercept = y_mean - slope * t_mean

    trend = slope * t + intercept
    return da - trend

## ERA5 hourly to daily

In [12]:
station_names = ['Guam', 'Yuma_PG' ,'Fort_Bragg'] # 'Pituffik', 'Fairbanks'

for stn in station_names:
    base_dir = f'/glade/derecho/scratch/ksha/EPRI_data/METRICS_STN/{stn}/'

    # ========================== #
    # get data
    list_ds = []
    for year in range(1958, 2025):
        fn = f'/glade/campaign/ral/hap/ksha/EPRI_data/ERA5_daily/{stn}_{year}.zarr'
        ds = xr.open_zarr(fn)
        list_ds.append(ds)
    ds_all = xr.concat(list_ds, dim='time')
    
    # ========================== #
    # get anomaly
    ds_all_anom = ds_all.copy()
    vars_ = list(ds_all.keys())
    for v in vars_:
        ds_all_anom[v] = ds_all_anom[v] - ds_all_anom[v].mean(['time'])
    ds_all_anom = ds_all_anom[vars_]
    
    # ========================== #
    # get detrend data
    ds_all_detrend = ds_all.copy()
    vars_ = list(ds_all.keys())
    for v in vars_:
        ds_all_detrend[v] = detrend_linear(ds_all[v], dim="time")
    ds_all_detrend = ds_all_detrend[vars_]
    
    # ======================= #
    # metrics
    ds_group = ds_all.groupby("time.year")
    ds_max  = ds_group.max(dim="time",  skipna=True)
    ds_min  = ds_group.min(dim="time",  skipna=True)
    ds_mean  = ds_group.mean(dim="time",  skipna=True)
    ds_30d = ds_group.map(
        lambda x: x.rolling(time=30, min_periods=30).mean().max(dim="time", skipna=True)
    )
    ds_min = ds_min.rename({'TREFHTMN': 'TREFHTMN_min'})[['TREFHTMN_min',]]
    ds_max = ds_max.rename({'PRECT': 'PRECT_max', 'TREFHTMX': 'TREFHTMX_max'})[['PRECT_max', 'TREFHTMX_max']]
    ds_30d = ds_30d.rename({'TREFHTMX': 'TREFHTMX_30d', 'PRECT': 'PRECT_30d'})[['TREFHTMX_30d', 'PRECT_30d']]
    ds_mean = ds_mean.rename({'PRECT': 'PRECT_mean', 'TREFHT': 'TREFHT_mean'})[['PRECT_mean', 'TREFHT_mean']]
    ds_metrics = xr.merge([ds_min, ds_max, ds_30d, ds_mean])
    ds_metrics = ds_metrics.rename({v: f"{v}_default" for v in ds_metrics.data_vars})

    # ========================== #
    # anomaly metrics
    ds_group = ds_all_anom.groupby("time.year")
    ds_max  = ds_group.max(dim="time",  skipna=True)
    ds_min  = ds_group.min(dim="time",  skipna=True)
    ds_mean  = ds_group.mean(dim="time",  skipna=True)
    ds_30d = ds_group.map(
        lambda x: x.rolling(time=30, min_periods=30).mean().max(dim="time", skipna=True)
    )
    ds_min = ds_min.rename({'TREFHTMN': 'TREFHTMN_min'})[['TREFHTMN_min',]]
    ds_max = ds_max.rename({'PRECT': 'PRECT_max', 'TREFHTMX': 'TREFHTMX_max'})[['PRECT_max', 'TREFHTMX_max']]
    ds_30d = ds_30d.rename({'TREFHTMX': 'TREFHTMX_30d', 'PRECT': 'PRECT_30d'})[['TREFHTMX_30d', 'PRECT_30d']]
    ds_mean = ds_mean.rename({'PRECT': 'PRECT_mean', 'TREFHT': 'TREFHT_mean'})[['PRECT_mean', 'TREFHT_mean']]
    ds_metrics_anom = xr.merge([ds_min, ds_max, ds_30d, ds_mean])
    ds_metrics_anom = ds_metrics_anom.rename({v: f"{v}_anom" for v in ds_metrics_anom.data_vars})

    # ========================== #
    # detrended metrics
    ds_group = ds_all_detrend.groupby("time.year")
    ds_max  = ds_group.max(dim="time",  skipna=True)
    ds_min  = ds_group.min(dim="time",  skipna=True)
    ds_mean  = ds_group.mean(dim="time",  skipna=True)
    ds_30d = ds_group.map(
        lambda x: x.rolling(time=30, min_periods=30).mean().max(dim="time", skipna=True)
    )
    ds_min = ds_min.rename({'TREFHTMN': 'TREFHTMN_min'})[['TREFHTMN_min',]]
    ds_max = ds_max.rename({'PRECT': 'PRECT_max', 'TREFHTMX': 'TREFHTMX_max'})[['PRECT_max', 'TREFHTMX_max']]
    ds_30d = ds_30d.rename({'TREFHTMX': 'TREFHTMX_30d', 'PRECT': 'PRECT_30d'})[['TREFHTMX_30d', 'PRECT_30d']]
    ds_mean = ds_mean.rename({'PRECT': 'PRECT_mean', 'TREFHT': 'TREFHT_mean'})[['PRECT_mean', 'TREFHT_mean']]
    ds_metrics_detrend = xr.merge([ds_min, ds_max, ds_30d, ds_mean])
    ds_metrics_detrend = ds_metrics_detrend.rename({v: f"{v}_detrend" for v in ds_metrics_detrend.data_vars})

    # ========================== #
    # save
    ds_final = xr.merge([ds_metrics, ds_metrics_anom, ds_metrics_detrend])
    save_name = base_dir + 'metrics.zarr'
    ds_final.to_zarr(save_name)
    print(save_name)

/glade/derecho/scratch/ksha/EPRI_data/METRICS_STN/Guam/metrics.zarr
/glade/derecho/scratch/ksha/EPRI_data/METRICS_STN/Yuma_PG/metrics.zarr
/glade/derecho/scratch/ksha/EPRI_data/METRICS_STN/Fort_Bragg/metrics.zarr


In [13]:
ds = xr.open_zarr('/glade/derecho/scratch/ksha/EPRI_data/METRICS_STN/Pituffik/metrics.zarr')

In [14]:
ds

<xarray.Dataset>
Dimensions:               (year: 67)
Coordinates:
    latitude              float32 ...
    longitude             float32 ...
  * year                  (year) int64 1958 1959 1960 1961 ... 2022 2023 2024
Data variables: (12/21)
    PRECT_30d_anom        (year) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    PRECT_30d_default     (year) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    PRECT_30d_detrend     (year) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    PRECT_max_anom        (year) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    PRECT_max_default     (year) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    PRECT_max_detrend     (year) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ...                    ...
    TREFHTMX_max_anom     (year) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    TREFHTMX_max_default  (year) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    TREFHTMX_max_detrend  (year) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    TREFHT_mean_anom      (year) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    TREFHT_mean_default   (year) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    TREFHT_mean_detrend   (year) float64 dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    last_updated:           2025-08-27 01:56:13.866333+00:00
    valid_time_start:       1940-01-01
    valid_time_stop:        2025-04-30
    valid_time_stop_era5t:  2025-08-21